<a href="https://colab.research.google.com/github/GreihMurray/NLP-3/blob/Super_Murray/supervised.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [67]:
import csv
from sklearn.tree import DecisionTreeClassifier
from sklearn.feature_extraction import DictVectorizer
from sklearn.pipeline import Pipeline
from tqdm import tqdm
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Embedding, Dense, TimeDistributed, Dropout, Bidirectional, Reshape
from tensorflow.keras.optimizers import SGD
import numpy as np
import tensorflow as tf
from sklearn.preprocessing import LabelEncoder
from keras.models import load_model
import joblib
from sklearn.metrics import accuracy_score
import pickle
import nltk
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.sequence import pad_sequences
import json

In [68]:
#/content/gdrive/MyDrive/Colab_Notebooks/NLP/kreole/word2indextest.json | /content/gdrive/MyDrive/Colab_Notebooks/NLP/kreole/tag2indextest.json PAIR WITH LOW_DIM
#/content/gdrive/MyDrive/Colab_Notebooks/NLP/kreole/word2indexdectree.json | /content/gdrive/MyDrive/Colab_Notebooks/NLP/kreole/tag2indextestdectree.json PAIR WITH DEC TREE


word2index_file = "/content/gdrive/MyDrive/Colab_Notebooks/NLP/kreole/word2indexdectree.json"
tag2index_file = "/content/gdrive/MyDrive/Colab_Notebooks/NLP/kreole/tag2indextestdectree.json"

In [69]:
from google.colab import drive

drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


Original

In [70]:
def read_file_to_sents():
    all_data = []
    with open("/content/gdrive/MyDrive/Colab_Notebooks/NLP/kreole/train.tsv", encoding="utf-8") as file:
        f = csv.reader(file, delimiter="\t")
        for line in tqdm(f, desc="Reading data..."):
            word = line[0]
            graphemes = line[1].split('-')

            cur_word = []

            for i in range(0, len(graphemes)):
                for j in range(0, len(graphemes[i])):
                    if j == 0:
                        cur_word.append((graphemes[i][j], 'B'))
                    else:
                        cur_word.append((graphemes[i][j], 'I'))

            all_data.append(cur_word)

    return all_data

From Challenge 2 https://github.com/GreihMurray/NLP-2

In [71]:
def pad(data):
  vocab = list(set([w for sent in data for (w,t) in sent]))
  vocab.append('<PAD>')
  tags = list(set([t for sent in data for (w,t) in sent]))
  tags.append('<PAD>')

  return vocab, tags

Combination of code from Challenge 2 (https://github.com/GreihMurray/NLP-2) and custom

In [72]:
def encode(vocab, tags, data, load=False):
  max_len = max([len(i) for i in data])

  word2index = {}
  tag2index = {}

  if load is False:
      word2index = {w: i for i, w in enumerate(vocab)}
      tag2index = {t: i for i, t in enumerate(tags)}
  else:
      with open(word2index_file) as infile:
          word2index = json.load(infile)  

      with open(tag2index_file) as outfile:
          tag2index = json.load(outfile)

  onehot = [[word2index[w[0]] for w in s] for s in data]
  X = pad_sequences(maxlen=max_len, sequences=onehot, padding="post", value=len(vocab)-1)  

  onehot_y = [[tag2index[w[1]] for w in s] for s in data]
  y = pad_sequences(maxlen=max_len, sequences=onehot_y, padding="post", value=tag2index["<PAD>"])
  y = to_categorical(y, num_classes=len(tags))

  # Used for saving word2index and tag2index in order to encode additional data in the same manner
  # Currently commented out due to issues with loading model
  with open(word2index_file, "w") as outfile:
    json.dump(word2index, outfile)

  with open(tag2index_file, "w") as outfile:
    json.dump(tag2index, outfile)

  return X, y, max_len

Code from Challenge 2 & Dr. Scannell (https://github.com/GreihMurray/NLP-2, 

In [73]:
def seq_model(data):
  #Original
    vocab, tags = pad(data)

    x, y, max_len = encode(vocab, tags, data)

    X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.1)
  
  
  # Dr. Scannell
    model = Sequential()
    model.add(Embedding(input_dim=len(vocab), output_dim=10, input_length=max_len))
    model.add(Bidirectional(LSTM(units=100, return_sequences=True, recurrent_dropout=0.05)))
    model.add(TimeDistributed(Dense(len(tags), activation="softmax")))
    model.compile(optimizer="adam", loss="poisson", metrics=["accuracy"])
  # From https://towardsdatascience.com/hyperparameter-tuning-with-kerastuner-and-tensorflow-c4a4d690b31a
    stop_early = tf.keras.callbacks.EarlyStopping(monitor='val_accuracy', patience=5)

    print("[INFO] training network...")
    sgd = SGD(0.05)
    history = model.fit(X_train, y_train, batch_size=32, epochs=50, validation_split=0.15, verbose=1, callbacks=stop_early)

    return model, X_test, y_test

Original

In [74]:
def eval_model(model, x_test, y_test):
    eval = model.evaluate(x_test, y_test)
    print(eval)

Original

In [75]:
def supervised():
    data = read_file_to_sents()

    print(data[:5])

    model, x_test, y_test = seq_model(data)

    eval_model(model, x_test, y_test) # Eval sequential model

    model.save('/content/gdrive/MyDrive/Colab_Notebooks/NLP/kreole/adamPoisson32_seq_model_low_dim')

    new_model = load_model('/content/gdrive/MyDrive/Colab_Notebooks/NLP/kreole/adamPoisson32_seq_model_low_dim')

    eval_model(new_model, x_test, y_test)

Original

In [76]:
def load_and_eval_model():
    data = read_file_to_sents()

    vocab, tags = pad(data)

    x, y, max_len = encode(vocab, tags, data, load=True)

    new_model = load_model('/content/gdrive/MyDrive/Colab_Notebooks/NLP/kreole/adamPoisson32_seq_model_low_dim')

    evals = new_model.evaluate(x, y)

    print('Default Accuracy: ', (evals[1] * 100))

    custom_eval(new_model, x, y)

Original

In [77]:
def undo_encode(x, y):
    all_data = []
    all_words = []
    all_tags = []

    word2index = {}
    tag2index = {}

    with open(word2index_file) as infile:
          word2index = json.load(infile)  

    with open(tag2index_file) as outfile:
          tag2index = json.load(outfile)

    for word in x:
        cur_word = []
        for letter in word:
            if letter == 51:
                break
            true_letter = list(word2index.keys())[list(word2index.values()).index(letter)]
            cur_word.append(true_letter)

        all_words.append(''.join(cur_word))

    for tags in y:
        cur_tags = []
        for tag in tags:
            if tag[0] == 1:
                cur_tags.append('I')
            elif tag[1] == 1:
                cur_tags.append('B')
            else:
                continue

        all_tags.append(''.join(cur_tags))

    for i in range(0, len(all_words)):
        all_data.append((all_words[i], all_tags[i]))

    return all_data

Original

In [78]:
def undo_encode_y(y):
    all_tags = []

    word2index = {}
    tag2index = {}

    with open(word2index_file) as infile:
          word2index = json.load(infile)  

    with open(tag2index_file) as outfile:
          tag2index = json.load(outfile)

    for tags in y:
        cur_tags = []
        for tag in tags:
            if round(tag[0]) == 1:
                cur_tags.append('I')
            elif round(tag[1]) == 1:
                cur_tags.append('B')
            else:
                continue

        all_tags.append(''.join(cur_tags))

    return all_tags

Original

In [79]:
def calc_accuracy(preds, y_test):
    total_right = 0

    for i in range(0, len(preds)):
        if preds[i] == y_test[i]:
            total_right += 1

    accuracy = 100 * (total_right/len(preds))

    return accuracy

Original

In [80]:
def calc_precision(preds, y_test):
    true_pos = 0
    false_pos = 0

    for i in range(0, len(preds)):
        for j in range(0, len(preds[i])):
            if preds[i][j] == 'I' and y_test[i][j] == 'I':
                true_pos += 1
            if preds[i][j] == 'I' and y_test[i][j] == 'B':
                false_pos += 1

    if (true_pos + false_pos) == 0:
        return 0.01

    precision = 100 * (true_pos / (true_pos + false_pos))

    return precision

Original

In [81]:
def calc_recall(preds, y_test):
    true_pos = 0
    false_neg = 0

    for i in range(0, len(preds)):
        for j in range(0, len(preds[i])):
            if preds[i][j] == 'I' and y_test[i][j] == 'I':
                true_pos += 1
            if preds[i][j] == 'B' and y_test[i][j] == 'I':
                false_neg += 1

    if true_pos + false_neg == 0:
        return 0
        
    recall = 100 * (true_pos / (true_pos + false_neg))

    return recall

Original

In [82]:
def custom_eval(model, x, y):
    preds = model.predict(x)

    preds = undo_encode_y(preds)
    y_clean = undo_encode_y(y)

    acc = calc_accuracy(preds, y_clean)

    print("Custom calculated Accuracy: ", acc)

    prec = calc_precision(preds, y_clean)

    print("Precision: ", prec)

    recall = calc_recall(preds, y_clean)

    print("Recall: ", recall)

    fscore = (2 * (prec * recall)) / (prec + recall)

    print("Fscore: ", fscore)

In [83]:
def dec_tree_eval(preds, y_test):
    acc = calc_accuracy(preds, y_test)

    print("Custom calculated Accuracy: ", acc)

    prec = calc_precision(preds, y_test)

    print("Precision: ", prec)

    recall = calc_recall(preds, y_test)

    print("Recall: ", recall)

    fscore = (2 * (prec * recall)) / (prec + recall)

    print("Fscore: ", fscore)

Original

In [84]:
def to_graphemes(data):
    graph_data = []

    for word_pair in data:
        word = word_pair[0]
        grap = word_pair[1]

        cur_word = []

        for i in range(0, len(word)):
            if i == (len(word) - 1):
                cur_word.append(word[i])

            else:
                if grap[i+1] == 'I':
                    cur_word.append(word[i])
                else:
                    cur_word.append(word[i] + '-')

        graph_data.append((word, ''.join(cur_word)))

    return graph_data


In [85]:
def split_x_y(data):
    all_x = []
    all_y = []

    for word in data:
        cur_x = []
        cur_y = []
        for letter in word:
            all_x.append(letter[0])
            all_y.append(letter[1])
        

    return np.asarray(all_x), np.asarray(all_y)

In [66]:
def dec_tree():
    data = read_file_to_sents()

    x, y = split_x_y(data)

    x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=50)

    y_test_clean = y_test

    le = LabelEncoder()
    le.fit(x_train)
    x_train = le.transform(x_train)
    x_train = x_train.reshape(-1, 1)
    x_test = le.transform(x_test)
    x_test = x_test.reshape(-1, 1)

    yle = LabelEncoder()
    yle.fit(y_train)
    y_train = yle.transform(y_train)
    y_train = y_train.reshape(-1, 1)
    y_test = yle.transform(y_test)
    y_test = y_test.reshape(-1, 1)

    clf = Pipeline([
        ('classifier', DecisionTreeClassifier(criterion='entropy')),
    ], verbose=1)

    clf.fit(x_train, y_train)

    preds = clf.predict(x_test)


    clean_preds = yle.inverse_transform(preds)

    dec_tree_eval(clean_preds, y_test_clean)

In [95]:
def hmm_eval(model, test, model_type):
    acc = model.accuracy(test)

    print(model_type, " Accuracy: ", acc)

    prec = model.precision(test)

    print(model_type, ' Precision: ', prec)

    rec = model.recall(test)

    print(model_type, ' Recall: ', rec)

    fscore = (2 * (prec['I'] * rec['I'])) / (prec['I'] + rec['I'])

    print(model_type, ' FScore (I): ', fscore)

    print('\n\n')

In [91]:
def HMMs():
    data = read_file_to_sents()

    cutoff = int(.75 * len(data))
    train = data[:cutoff]
    test = data[cutoff:]

    uni_tag = nltk.UnigramTagger(train)
    hmm_eval(uni_tag, test, 'Unigram')

    bitag = nltk.BigramTagger(train)
    hmm_eval(bitag, test, 'Bigram')

    tritag = nltk.TrigramTagger(train)
    hmm_eval(tritag, test, 'Trigram')

In [96]:
HMMs()

Reading data...: 12812it [00:00, 139601.65it/s]


Unigram  Accuracy:  0.9711306406812076
Unigram  Precision:  {'B': 0.9994360131255127, 'I': 0.7464387464387464}
Unigram  Recall:  {'B': 0.9690296281566911, 'I': 0.9940379403794037}
Unigram  FScore (I):  0.8526266852626685



Bigram  Accuracy:  0.9785984244797595
Bigram  Precision:  {'B': 0.9993899339095068, 'I': 0.8000872981230903}
Bigram  Recall:  {'B': 0.9772320540862994, 'I': 0.9934959349593496}
Bigram  FScore (I):  0.8863636363636364



Trigram  Accuracy:  0.9785984244797595
Trigram  Precision:  {'B': 0.9993899339095068, 'I': 0.8000872981230903}
Trigram  Recall:  {'B': 0.9772320540862994, 'I': 0.9934959349593496}
Trigram  FScore (I):  0.8863636363636364





In [86]:
dec_tree()

Reading data...: 12812it [00:00, 74704.00it/s]


[Pipeline] ........ (step 1 of 1) Processing classifier, total=   0.0s
Custom calculated Accuracy:  97.15938376954355
Precision:  75.51637279596977
Recall:  99.33730947647449
Fscore:  85.80423583285634


Original

In [ ]:
load_and_eval_model()

Reading data...: 12812it [00:00, 24393.28it/s]


401/401 [==============================] - 5s 11ms/step - loss: 0.3335 - accuracy: 0.9999
Default Accuracy:  99.98655915260315
Custom calculated Accuracy:  99.77364970340305
Precision:  99.98243258504512
Recall:  99.97866894197952
Fscore:  99.98055072809291


Currently best performance is with output dim of 10, better than output dim of 50 by approx. 0.4%

Original

In [ ]:
supervised()

Reading data...: 12812it [00:00, 147155.67it/s]


[INFO] training network...
Epoch 1/50
307/307 [==============================] - 28s 73ms/step - loss: 0.4124 - accuracy: 0.9222 - val_loss: 0.3571 - val_accuracy: 0.9670
Epoch 2/50
307/307 [==============================] - 22s 71ms/step - loss: 0.3434 - accuracy: 0.9884 - val_loss: 0.3390 - val_accuracy: 0.9931
Epoch 3/50
307/307 [==============================] - 24s 78ms/step - loss: 0.3373 - accuracy: 0.9968 - val_loss: 0.3357 - val_accuracy: 0.9982
Epoch 4/50
307/307 [==============================] - 23s 74ms/step - loss: 0.3355 - accuracy: 0.9988 - val_loss: 0.3351 - val_accuracy: 0.9991
Epoch 5/50
307/307 [==============================] - 22s 72ms/step - loss: 0.3350 - accuracy: 0.9991 - val_loss: 0.3347 - val_accuracy: 0.9992
Epoch 6/50
307/307 [==============================] - 22s 72ms/step - loss: 0.3348 - accuracy: 0.9993 - val_loss: 0.3346 - val_accuracy: 0.9993
Epoch 7/50
307/307 [==============================] - 22s 73ms/step - loss: 0.3346 - accuracy: 0.9993 - val_l

41/41 [==============================] - 1s 12ms/step - loss: 0.3336 - accuracy: 0.9998
[0.3335980772972107, 0.9997833371162415]
